In [1]:
import sys
import os
import glob
from lxml import etree
import pandas as pd
import numpy as np


In [2]:
sys.path.append(os.path.abspath("./../"))

In [3]:
from fachreferats_functions import check_duplicates, extract_data



## Data

In [4]:
books_donated_1 = pd.read_excel("./../../data/input/Anfragen_Geschenk_Muster_Spendende_Beispiele.xlsx").dropna(how='all', axis=1)

In [5]:
books_donated_1

,Titel,Vorname_Autor,Nachname_Autor,Erscheinungsjahr,ISBN,Erscheinungsort,Verlag
0,The picture of Dorian Gray,Oscar,Wilde,1994,9.780141e+12,NaN,NaN
1,Gustav Klimt,NaN,NaN,2008,9.781403e+12,NaN,NaN
2,La familia de Pascual Duarte,Camilo José,Cela,2009,9.788423e+12,NaN,NaN
3,Auf der Eidechsburg,Ilse-Dore,Tanner,1938?,NaN,Leipzig,A. H. Payne
4,Bayerisches Kochbuch,Maria,Hofmann,1950,NaN,München,Birken
5,Centenaire de l‘Impressionisme,NaN,NaN,1974,NaN,Paris,Musées nationaux
6,Les parents terribles,Jean,Cocteau,1938,NaN,Paris,Gallimard
7,Straightforward Statistics,James D.,Evans,1996,NaN,Pacific Grove,Brooks/Cole


In [6]:
books_donated_1["ISBN"].dtype == float

True

In [7]:
books_donated_2 = pd.read_excel("./../../data/input/Anfragen_Geschenk_Muster_Spendende_Beispiele_mistakes.xlsx").dropna(how='all', axis=1)

In [8]:
books_donated_2 

,titel,Vorname_Autor,Nachname_Autor,Erscheinungsjahr,ISBN,Erscheinungsort,Verlag
0,The picture of Dorian Gray,Oscar,Wilde,1994,'978-0140623222,NaN,NaN
1,Gustav Klimt,NaN,NaN,2008,978-1402759208,NaN,NaN
2,La familia de Pascual Duarte,Camilo José,Cela,2009,978 8423339044,NaN,NaN
3,Auf der Eidechsburg,Ilse-Dore,Tanner,1938?,NaN,Leipzig,A. H. Payne
4,Bayerisches Kochbuch,Maria,Hofmann,1950,NaN,München,Birken
5,Centenaire de l‘Impressionisme,NaN,NaN,1974,NaN,Paris,Musées nationaux
6,Les parents terribles,Jean,Cocteau,1938,NaN,Paris,Gallimard
7,Straightforward Statistics,James D.,Evans,1996,NaN,Pacific Grove,Brooks/Cole


In [9]:
books_donated_2[" ISBN"].dtypes == "object"

True

In [10]:
books_donated_3 = pd.read_excel("./../../data/input/Anfragen_Geschenk_Muster_Spendende_Beispiele_only_isbn.xlsx").dropna(how='all', axis=1)

In [11]:
books_donated_3

,Titel,Vorname_Autor,Nachname_Autor,Erscheinungsjahr,ISBN
0,The picture of Dorian Gray,Oscar,Wilde,1994,9780140623222
1,Gustav Klimt,NaN,NaN,2008,9781402759208
2,La familia de Pascual Duarte,Camilo José,Cela,2009,9788423339044


In [12]:
books_donated_3["ISBN"]

0    9780140623222
1    9781402759208
2    9788423339044
Name: ISBN, dtype: int64

In [13]:
# Wenn die ISBNs der Tabelle als Zahl mit Exponent formatiert ist, nutze:
books_donated["ISBN"] = books_donated["ISBN"].fillna(0).astype('Int64').astype(str)

# Wenn die ISBNs der Tabelle als Text formatiert ist, nutze:
# books_donated["ISBN"] = books_donated["ISBN"].astype(str)


NameError: name 'books_donated' is not defined

In [63]:
"A".lower()

'a'

In [68]:
def identify_ISBN_column(df):
    if "ISBN" not in df.columns.tolist() and len([column for column in df.columns.tolist() if "isbn" in column.lower()]) > 1:
        print("Your table contains several columns for ISBN and none of them is just called 'ISBN'. Please, open the table in Excel and Calc and the column that you want to use for the checking 'ISBN', and read the table again with your Jupyter Notebook. Otherwise it won't work.")


def strip_columns(df):
    if " " in [column[0] for column in df.columns.tolist()] or " " in [column[-1] for column in df.columns.tolist()]:
        print("One column has whitespace before or after the name of the column. This white space is going to be deleted.")
        for column in df.columns.tolist():
            if column[0] == " " or column[-1] == " ":
                print("Deleting white space in '" + column + "'")
                df.rename(columns = {column: column.strip()}, inplace = True)
    return df

def try_to_find_missing_columns(df, mandatory_columns):
    for mantadory_column_name, mantadory_column_alternatives  in mandatory_columns.items():
        if mantadory_column_name not in df.columns.tolist():
            print("The column ", mantadory_column_name, " is mandatory but missing in the table. We are going to try to find another one in the table that contains this information, but perhaps this creates errors!")
            for mantadory_column_alternative in mantadory_column_alternatives:
                if mantadory_column_alternative in df.columns.tolist():
                    print("Replacing ", mantadory_column_alternative, " with ", mantadory_column_name)
                    df.rename(columns = {mantadory_column_alternative: mantadory_column_name}, inplace = True)
    return df


def convert_ISBN_to_str(df):
    if df["ISBN"].dtype == float:
        print("Changing ISBN type to float to str")
        df["ISBN"] = df["ISBN"].fillna(0).astype('Int64').astype(str)
        
    if df["ISBN"].dtype == int or df["ISBN"].dtype == "int64":
        print("Changing ISBN type to int to str")
        df["ISBN"] = df["ISBN"].astype(str)
    return df

def clean_ISBN(df):
    df["ISBN"] = df["ISBN"].str.replace(r"\D", "", regex=True)
    return df

def clean_data(
    df,
    mandatory_columns =
        {
            "Titel" : ["titel", "title"],
            "Vorname_Autor" : ["vorname_autor", "vorname", "Vorname"],
            "Nachname_Autor": ["nachname_autor", "author", ],
            "Erscheinungsjahr" : ["erscheinungsjahr", "jahr"],
            "ISBN" : ["isbn", "isbn-10", "ISBN-10"]
        },
    ):

    identify_ISBN_column(df)

    df = strip_columns(df)

    df = try_to_find_missing_columns(df, mandatory_columns)

    df = convert_ISBN_to_str(df)

    df = clean_ISBN(df)

    return df
    


In [72]:
books_donated_1 = clean_data(books_donated_1)

In [73]:
books_donated_1

In [70]:
clean_data(books_donated_2)

In [71]:
books_donated_3

,Titel,Vorname_Autor,Nachname_Autor,Erscheinungsjahr,ISBN
0,The picture of Dorian Gray,Oscar,Wilde,1994,9780140623222
1,Gustav Klimt,NaN,NaN,2008,9781402759208
2,La familia de Pascual Duarte,Camilo José,Cela,2009,9788423339044


In [67]:
clean_data(books_donated_3)

Changing ISBN type to int to str


,Titel,Vorname_Autor,Nachname_Autor,Erscheinungsjahr,ISBN
0,The picture of Dorian Gray,Oscar,Wilde,1994,9780140623222
1,Gustav Klimt,NaN,NaN,2008,9781402759208
2,La familia de Pascual Duarte,Camilo José,Cela,2009,9788423339044
